(c) Juan Gomez 2019. Thanks to Universidad EAFIT for support. This material is part of the course Introduction to Finite Element Analysis

# Numerical Integration in The Finite Element Method

## Introduction

This Notebook covers a fundamental application of numerical integration within the context of the finite element method. We use Gaussian quadratures to integrate functions over domains corresponding to finite elements: here we cover in particularl the case of a bi-linear element. The notebook ends with a propossed activity which requires extending the given subroutines to compute energy terms typically appearing in finite element methods for plane elasticity. **After completing this notebook you should be able to:**


* Undesrtand the mathematical basis of a general quadrature.

* Undesrtand the relationship between the physical and the natural space in the formulation of finite elements.

* Transform functions and elements from the physical to the natural space in order to perform numerical integration over arbitrary geometries.

## Quadrature rules in the FEM.

In the finite element method we are interested in finding integrals like:

$$I=\int_{V(\overrightarrow x)}f(\overrightarrow x)\operatorname d{V(\overrightarrow x)}$$

over the domain $V(\overrightarrow x)$ of typical finite elements (see figure below).


<center><img src="img/physical.png" alt="files" style="width:300px"></center>

In a general finite element mesh both the integrand and the integration domain are expected to change from element-to-element then the integration must me performed using efficient numerical integration schemes or **quadratures** that can handle the differences from one element to the other.

A **quadrature** is an approximation of an integral by a weighted superposition of functional evaluations:

$$I=\int_{V(\overrightarrow x)}f(\overrightarrow x)\operatorname d{V(\overrightarrow x)\approx\sum_{i=1}^{Npts}f(\overrightarrow{x_i})w_i}.$$

In this expression the integral is approximated by a summation of products $f(\overrightarrow x_i) w_i$ of function evaluations at **integration points** with position vector $\overrightarrow x_i$ multiplied by **weighting factors** $w_i$ associated to each evaluation point $i$.

In the finite element method the most commonly used numerical quadratures correspond to **Gaussian** integration formulas where the evaluation points and weighting factors are selected for maximum accuracy. In Gaussian schemes a quadrature with $n$ integration points integrates exactly polynomial functions of order $2n-1$ (see Bathe, 2006 and Press et al, 2006.)

### The physical and the canonical space.

Since in the finite element method the domain of integration changes from element-to-element the most efficiet method to conduct the numerical integration is to transform (or mapp) each element from the **physical space** into a canonical element. The fictitous or methematical space of the canonical element is termed the **natural space**.

The transformation is ilustrated in the figure which shows a typical bi-lineal two-dimensional finite element taken from a mesh of an arbitrary computational domain and the equivalent canonical element. The black dots represent the **nodes** of the element. 

<center><img src="img/mapping.png" alt="files" style="width:500px"></center>

The space in which the element has been created during the meshing process is denoted by the position vector $\overrightarrow x$ and is termed the **physical space**. Notice that in the **physical space** the element has an arbitrary shape, distorted with respect to the perfect square representation in the **natural space**.

The fact that the element in the **natural space** does not change (notice in this case that it is a perfect square of side $2.0$) implies that the **shape functions** and location of **integration points** are always the same and therefore can be implemented in a computer code. For instance, in the figure above the points marked with a cross correspond to evaluation points for a 2X2 Gauss **quadrature**.

### Element mapping.

Transformation of the element from the **physical** to the **natural** space requires also transformation of the integrand in $I$. This process is equivalent to a change of variables.

First, the transformation between both spaces can be written in the general form:

$$x_i=x_i(r_J)$$
$$r_J=r_J(x_i)$$

where $x_i$ and $r_I$ represent the position vectors of a point in the physical and natural space respectively.

In the finite element method the above functional relationships are formulated in terms of the element shape functions $N_i^Q(r_J)$ as:

$$x_i=N_i^Q(r_J)x^Q$$

which implies that the geometry of the element, or equivalently the independent variable in the functions, is also being approximated via interpolation.

This relation between spaces allows us to transform the function $f(\overrightarrow x)$ from the **physical** to the **natural space** as indicated by the general expression:

$$f=f(x_i)\equiv f\left[x_i(r_J)\right]\equiv F(r_J).$$


To transform the differential volume element $dV(\overrightarrow x)$ in the **physical space** into the differential volume element $dV(\overrightarrow r)$ in the **natural space**, the transformation can be understood like the result of an actual deformation process taking the element from one configuration to the other. This transofrmation can be described in terms of Nanson's relation from continuum mechanics (see Malvern, 1969) like:

$$dV(\overrightarrow x)=\left|J(\overrightarrow r)\right|dV(\overrightarrow r)$$

where $J(\overrightarrow r)$ is the Jacobian of the transformation:


$$J_{iJ}=\frac{\partial x_i}{\partial r_J}$$

while $\left|J(\overrightarrow r)\right|$ is its determinant.


Substitution of $F(\overrightarrow r)$ and $\left|J(\overrightarrow r)\right|dV(\overrightarrow r)$ in $I$ gives:


$$I=\int_{V(\overrightarrow x)}f(\overrightarrow x)dV(\overrightarrow x)\equiv\int_{V(\overrightarrow r)}F(\overrightarrow r)\left|J(\overrightarrow r)\right|dV(\overrightarrow r)$$

and therefore its evaluation is also possible in terms of the **quadrature** :


$$I=\int_{V(\overrightarrow r)}F(\overrightarrow r)\left|J(\overrightarrow r)\right|dV(\overrightarrow r)\approx\sum_{I\;=1}^{Npts}F(\overrightarrow{r_I})\left|J(\overrightarrow{r_I})\right|w_I.$$



Notice that the Jacobian matrix $J$ carries with it all the information about the change of form in the element when going from the **physical** to the **natural space**. This tensor however can easily be evaluated at each integration point using the relation between spaces in terms of the shape functions and the nodal coordinates. For instance, the contribution from the $Q$ nodal point is:

$$J_{iJ}=\frac{\partial x_i}{\partial r_J}\equiv\frac{\partial N_i^Q}{\partial r_J}x^Q.$$
  

**Note:** In the particular case of a quadrilateral element the integral in the natural space takes the form:

$$I=\int_{r=-1}^{r=+1}\int_{s=-1}^{s=+1}F(r , s)\left|J\right|drds$$
 
and the **qudrature** becomes an straightforward application of two iterated inetgrals.

## Finding the area of a bi-lineal element.

Use a 2x2 Gauss **quadrature** to compute the area of the quadrilaterla bi-lineal finite element shown in the figure.

<center><img src="img/trape.png" alt="files" style="width:200px"></center>


The coordinates of the element nodes in the physical space are stored in the following array


$$coord=\begin{bmatrix}0&0\\1&0\\1&2\\0&1\end{bmatrix}$$

In this particular case $f(\overrightarrow x) = 1.0$ and the computational domain is the element surface $S(\overrightarrow x)$ so the integral reads:

$$I=\int_{S(\overrightarrow x)}\operatorname d{S(\overrightarrow x)}.$$

Transformation to the **natural** space yields:

$$I=\int_{r=-1}^{r=+1}\int_{s=-1}^{s=+1}\left|J\right|drds.$$

Note that all the information about the change in geometry is contained in the determinant of the Jacobian matrix. In this problem the Jacobian matrix and its determinant will be found numerically.

### Python solution.
The main program loops through all the Gauss points and at each point it proceeds as follows:

* Evaluates the Jacobian determinant.
* Evaluates the function (already written in the natural space).
* Accumulates the value of the product $F(\overrightarrow{r_i})\left|J(\overrightarrow{r_i})\right|w_i$ in the variable $fun$:

$$fun\leftarrow fun+ F(\overrightarrow{r_i})\left|J(\overrightarrow{r_i})\right|w_i$$


For the Python implementation we start, as usual, by importing the required modules. In this case we only need to import `numpy`. 

In [1]:
%matplotlib inline        
import numpy as np

Subroutine `gpoints2x2()` creates arrays `xp[]` and `xw[]` storing the location of the evaluation points in the Gauss quadrature and the corresponding weighting factors. Notice that the coordinates of the integration points correspond to the iterated application of a one-dimensional 2-point quadrature. The weighting factors are obtained from products of the factor from each direction.

**Questions:**

**Document the following subroutine by adding a definition of each input and output parameter indicating the type of data in each case and the definition of the parameter within the current context.**

In [2]:
def gpoints2x2():
    """Gauss points for a 2 by 2 grid

    """
    xw = np.zeros([4])
    xp = np.zeros([4, 2])
    xw[:] = 1.0
    xp[0, 0] = -0.577350269189626
    xp[1, 0] = 0.577350269189626
    xp[2, 0] = -0.577350269189626
    xp[3, 0] = 0.577350269189626

    xp[0, 1] = 0.577350269189626
    xp[1, 1] = 0.577350269189626
    xp[2, 1] = -0.577350269189626
    xp[3, 1] = -0.577350269189626

    return xw, xp

Subroutine `stdm4NQ()` receives as input parameters the nodal coordinates of the 4 nodal points of the element and the coordinates of a Gauus point. The subroutine evaluates the shape function derivatives $\frac{\partial N_i^Q}{\partial r_J}$ at the Gauss point and use it in the evaluation of the Jacobian and its determinant.

**Questions:**

**Document the following subroutine by adding a definition of each input and output parameter indicating the type of data in each case and the definition of the parameter within the current context.**

In [3]:
def stdm4NQ(r, s, coord):
    """Shape function derivatives for a 4-noded quad element

    Parameters
    ----------
   

    Returns
    -------

    """
    nn = 4
    dhdx = 0.25*np.array([
            [s - 1, -s + 1, s + 1, -s - 1],
            [r - 1, -r - 1, r + 1, -r + 1]])
    det  = jacoper(dhdx, coord)

    return det

The contribution from nodal point $Q$ to the Jacobian of the transformation takes the form:

$$
\begin{array}{l}\begin{bmatrix}\frac{\partial x}{\partial r}&\frac{\partial y}{\partial r}\\\frac{\partial x}{\partial s}&\frac{\partial y}{\partial s}\end{bmatrix}=\begin{bmatrix}\cdots&\begin{array}{c}\frac{\partial N^Q}{\partial r}\\\frac{\partial N^Q}{\partial s}\end{array}&\cdots\end{bmatrix}\begin{bmatrix}\vdots\\\begin{array}{cc}x^Q&y^Q\end{array}\\\vdots\end{bmatrix}\\\end{array}
$$


where $N^Q$ is the shape function associated to node $Q$ and $x^Q$ and $y^Q$ are the nodal coordinates of the $Q$ point.

Subroutine `jacoper()` uses the shape function derivatives and the coordinates of the nodal points to compute the determinant of the Jacobian matrix.


**(Add comments to clarify the relevant steps ion the code below)**.


In [4]:
def jacoper(dhdx, coord):
    """
    Compute the Jacobian and the determinant for the transformation evaluated at
    the Gauss point.

    Parameters
    ----------
    dhdx : ndarray
      Derivatives of the interpolation function with respect to the
      natural coordinates.
    coord : ndarray
      Coordinates of the nodes of the element (nn, 2).

    Returns
    -------
    det : float.
      Determinant of the Jacobian evaluated at `(r, s)`.

    """
    jaco = dhdx.dot(coord)
    det = np.linalg.det(jaco)
    
    return det

The part of the function resulting from direct transformation of $f(x , y)$ is defined in the simple subroutine `myfun()`.

In [5]:
def myfunc(r , s):
    """
    """
    FR = 1.0
    
    return FR

In [6]:
nnodes = 4
coord = np.zeros([nnodes, 2])
coord =([0.0 , 0.0], [1.0 , 0.0], [1.0 , 2.0], [0.0 , 1.0])

In [7]:
fun = 0.0
XW , XP = gpoints2x2()
ngpts = 4
for i in range(0, ngpts):
    ri  = XP[i, 0]
    si  = XP[i, 1]
    alf = XW[i]
    ddet = stdm4NQ(ri, si, coord)
    fun = fun + myfunc(ri , si)*alf*ddet
print(fun)

1.5000000000000002


**Questions:**

**1) Assume that $F(r , s) = r^2+s^2$ is the function that results after a prior transformation from the physical to the natuarl space. Compute the value of $I$**

$$I=\int_{r=-1}^{r=+1}\int_{s=-1}^{s=+1}(r^2+s^2)\left|J\right|drds$$

**over the element discussed in the notebook.**


**2) Change the geometry of the element, considering:(i) a perfect square (ii) a rectangle (iii) a trapezoid and (iv) a general distorted element in the physical space. For each one of these elements identify how these changes modify the Jacobian matriz. (Note: Try computing the inverse of the Jacobian)**

### Glossary of terms

**Quadrature:** Simple summation formula to approximate an integral after evaluating the integrand at a pre-defined set of points and weighting by different factors.

**Gauss points:** Evaluation points for a specific qudrature where the points and weighting factors are selected to provide maximum accuracy.

**Physical space:** Space of a finite element as created in a meshing process of a coputational domain.

**Natural space:** Matematical space to which all the elements of a mesh are transformedin order to facilitate interpolatio and numerical integration.


### Class activity
In several problems of theory of elastcity it is sometimes required to compute terms like 

$$I\;=\;\int_{V(\overrightarrow x)}2\mu\varepsilon_{xx}^2\operatorname dV(\overrightarrow x)+\int_{V(\overrightarrow x)}2\mu\varepsilon_{yy}^2\operatorname dV(\overrightarrow x)$$

contributing to the strain energy of the system and where


$$\varepsilon_{xx}=\frac{\partial u}{\partial x}$$

and

$$\varepsilon_{yy}=\frac{\partial v}{\partial x}$$

with $u$ and $v$ being the horizontal and vertical components of the displacement vector respectively.

Modify the subroutines provided in the **Notebook** to compute the integral $I$ assuming that it is to be obtained for a bi-lineal finite element with known nodal displacements.

### References

* Bathe, Klaus-Jürgen. Finite element procedures. Klaus-Jurgen Bathe, 2006.

* Press, William H., et al. Numerical recipes in Fortran 90. Vol. 2. Cambridge: Cambridge university press, 1996.

* Malvern, Lawrence E. Introduction to the Mechanics of a Continuous Medium. No. Monograph. 1969.

In [8]:
from IPython.core.display import HTML
def css_styling():
    styles = open('./nb_style.css', 'r').read()
    return HTML(styles)
css_styling()